# Get & Clean Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
sns.set()

In [2]:
# Download data
data = yf.download("BA SPY", start="2020-03-31", end="2020-10-01")
stocks = data['Adj Close'][:]
stocks

[*********************100%***********************]  2 of 2 completed


In [3]:
ba = data['Adj Close'][:]
ba

,BA,SPY
Date,,
2020-03-31,149.139999,253.681808
2020-04-01,130.699997,242.264893
2020-04-02,123.269997,247.855255
2020-04-03,124.519997,244.272705
2020-04-06,148.770004,260.679565
...,...,...
2020-09-24,146.050003,321.075287
2020-09-25,156.029999,326.266083
2020-09-28,166.080002,331.685181


In [12]:
# Calculate data from prices
for col in ba.columns:
    # Daily Returns for each column
    ba[f'{col}_ret'] = ba[col].diff(1)

    # Starting value normalization for each col
    start_val_price = ba[col].iloc[0]
    start_val_ret = ba[f'{col}_ret'].iloc[1]
    
    ba[f'norm_{col}_price'] = ba[col].div(start_val_price).mul(100)
    ba[f'norm_{col}_ret'] = ba[f'{col}_ret'].div(start_val_ret).mul(100)
    
    # Smooth returns with 7 day rolling average for normalized daily returns
    ba[f'7d_roll_avg_norm_{col}_price'] = ba[f'norm_{col}_price'].rolling(window=7).mean()
    ba[f'7d_roll_avg_norm_{col}_ret'] = ba[f'norm_{col}_ret'].rolling(window=7).mean()

ba.head(10)

,BA,SPY,BA_ret,norm_BA_price,norm_BA_ret,7d_roll_avg_norm_BA_price,7d_roll_avg_norm_BA_ret,SPY_ret,norm_SPY_price,norm_SPY_ret,7d_roll_avg_norm_SPY_price,7d_roll_avg_norm_SPY_ret
Date,,,,,,,,,,,,
2020-03-31,149.139999,253.681808,NaN,100.000000,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN
2020-04-01,130.699997,242.264893,-18.440002,87.635777,100.000000,NaN,NaN,-11.416916,95.499513,100.000000,NaN,NaN
2020-04-02,123.269997,247.855255,-7.430000,82.653880,40.292838,NaN,NaN,5.590363,97.703204,-48.965610,NaN,NaN
2020-04-03,124.519997,244.272705,1.250000,83.492019,-6.778741,NaN,NaN,-3.582550,96.290982,31.379315,NaN,NaN
2020-04-06,148.770004,260.679565,24.250008,99.751914,-131.507616,NaN,NaN,16.406860,102.758478,-143.706589,NaN,NaN
2020-04-07,141.580002,260.945343,-7.190002,94.930939,38.991331,NaN,NaN,0.265778,102.863246,-2.327928,NaN,NaN
2020-04-08,146.869995,269.704834,5.289993,98.477937,-28.687595,92.420352,NaN,8.759491,106.316190,-76.723793,100.204516,NaN
2020-04-09,151.839996,273.809021,4.970001,101.810377,-26.952281,92.678978,-2.091723,4.104187,107.934039,-35.948299,101.337950,-25.184701
2020-04-13,147.330002,271.309113,-4.509995,98.786377,24.457668,94.271921,-12.883485,-2.499908,106.948588,21.896530,102.973532,-36.342339


In [74]:
# Load COVID Data
covid_data = pd.read_csv('./all-states-history.csv', parse_dates=True, index_col=0)
covid_data

,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
date,,,,,,,,,,,,,,,,,,,,,
2020-10-02,WY,B,53.0,NaN,0,NaN,280.0,280.0,32.0,6,...,103278.0,1983,NaN,NaN,NaN,NaN,102353.0,2095,165242.0,1454
2020-10-02,NE,A,493.0,NaN,15,NaN,2356.0,2356.0,227.0,7,...,464264.0,4722,NaN,NaN,NaN,NaN,464576.0,4731,634687.0,9733
2020-10-02,ND,B,191.0,188.0,0,3.0,902.0,902.0,111.0,18,...,626450.0,7119,9731.0,NaN,NaN,NaN,244312.0,1412,649885.0,7432
2020-10-02,NC,A+,3608.0,3579.0,29,29.0,NaN,NaN,921.0,0,...,3099704.0,36043,NaN,3063.0,NaN,NaN,NaN,0,3094417.0,35876
2020-10-02,MT,C,186.0,NaN,5,NaN,729.0,729.0,177.0,2,...,353362.0,4653,NaN,NaN,NaN,NaN,NaN,0,353362.0,4653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-24,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
2020-01-23,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
2020-01-23,MA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,2.0,1,NaN,NaN,NaN,NaN,NaN,0,2.0,1


In [30]:
covid = covid_data.groupby('date').sum()[['deathIncrease']]

start_date = '2020-03-31'
end_date = '2020-10-01'

covid = covid.loc[start_date:end_date]
covid

,deathIncrease
date,
2020-03-31,890
2020-04-01,1011
2020-04-02,1172
2020-04-03,1286
2020-04-04,1482
...,...
2020-09-27,307
2020-09-28,257
2020-09-29,739


In [37]:
# Create more data columns
for col in covid.columns:
    # Daily differences
    covid[f'{col}_diff'] = covid[col].diff(1)
    
    # Normalized
    first_non_null_val = covid[f'{col}_diff'].iloc[1]
    
    covid[f'norm_{col}_diff'] = covid[f'{col}_diff'].div(first_non_null_val).mul(100)
    
    # Norm Rolling
    covid[f'7d_roll_avg_norm_{col}'] = covid[f'norm_{col}_diff'].rolling(window=7).mean()
    
covid.head()

,death,positive,death_diff,norm_death_diff,7d_roll_avg_norm_death,positive_diff,norm_positive_diff,7d_roll_avg_norm_positive
date,,,,,,,,
2020-03-31,4313.0,198295.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-01,5324.0,224086.0,1011.0,100.000000,NaN,25791.0,100.000000,NaN
2020-04-02,6496.0,252146.0,1172.0,115.924827,NaN,28060.0,108.797643,NaN
2020-04-03,7782.0,284222.0,1286.0,127.200791,NaN,32076.0,124.368966,NaN
2020-04-04,9264.0,317434.0,1482.0,146.587537,NaN,33212.0,128.773603,NaN


In [38]:
df_comb = ba.join(covid)
df_comb

,BA,SPY,BA_ret,norm_BA_price,norm_BA_ret,7d_roll_avg_norm_BA_price,7d_roll_avg_norm_BA_ret,SPY_ret,norm_SPY_price,norm_SPY_ret,7d_roll_avg_norm_SPY_price,7d_roll_avg_norm_SPY_ret,death,positive,death_diff,norm_death_diff,7d_roll_avg_norm_death,positive_diff,norm_positive_diff,7d_roll_avg_norm_positive
Date,,,,,,,,,,,,,,,,,,,,
2020-03-31,149.139999,253.681808,NaN,100.000000,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,4313.0,198295.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-01,130.699997,242.264893,-18.440002,87.635777,100.000000,NaN,NaN,-11.416916,95.499513,100.000000,NaN,NaN,5324.0,224086.0,1011.0,100.000000,NaN,25791.0,100.000000,NaN
2020-04-02,123.269997,247.855255,-7.430000,82.653880,40.292838,NaN,NaN,5.590363,97.703204,-48.965610,NaN,NaN,6496.0,252146.0,1172.0,115.924827,NaN,28060.0,108.797643,NaN
2020-04-03,124.519997,244.272705,1.250000,83.492019,-6.778741,NaN,NaN,-3.582550,96.290982,31.379315,NaN,NaN,7782.0,284222.0,1286.0,127.200791,NaN,32076.0,124.368966,NaN
2020-04-06,148.770004,260.679565,24.250008,99.751914,-131.507616,NaN,NaN,16.406860,102.758478,-143.706589,NaN,NaN,11922.0,371936.0,1318.0,130.365974,NaN,29002.0,112.450079,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-24,146.050003,321.075287,-5.129990,97.928124,27.819897,105.989580,13.511001,0.853546,126.566146,-7.476153,128.887323,19.018114,194848.0,6942070.0,921.0,91.097923,73.322029,43772.0,169.718119,166.500496
2020-09-25,156.029999,326.266083,9.979996,104.619820,-54.121445,104.894730,8.854979,5.190796,128.612329,-45.465833,128.398882,10.853087,195692.0,6997596.0,844.0,83.481701,72.516603,55526.0,215.292156,170.953876
2020-09-28,166.080002,331.685181,10.050003,111.358457,-54.501094,104.755839,1.123332,5.419098,130.748508,-47.465515,128.381500,0.386212,197125.0,7117430.0,257.0,25.420376,73.632895,36524.0,141.615292,170.124684


In [39]:
import plotly.express as px

In [40]:
fig = px.line(df_comb)

fig.update_layout(dict(
    title="Prices, Avgs, Returns, Deaths, Positives"
))
fig.show()

In [42]:
cols = [
    '7d_roll_avg_norm_death',
    '7d_roll_avg_norm_positive',
    '7d_roll_avg_norm_BA_ret',
    '7d_roll_avg_norm_SPY_ret'
]

fig = px.line(df_comb[cols])

fig.update_layout(dict(
    title="Seven Day Rolling Avgs for Returns, Positives, and Deaths"
))
fig.show()

In [43]:
df_comb[cols].corr()

,7d_roll_avg_norm_death,7d_roll_avg_norm_positive,7d_roll_avg_norm_BA_ret,7d_roll_avg_norm_SPY_ret
7d_roll_avg_norm_death,1.000000,-0.343445,0.052743,-0.305913
7d_roll_avg_norm_positive,-0.343445,1.000000,0.283568,0.024351
7d_roll_avg_norm_BA_ret,0.052743,0.283568,1.000000,0.592298
7d_roll_avg_norm_SPY_ret,-0.305913,0.024351,0.592298,1.000000


In [44]:
## Testing for stationarity with DF test
import statsmodels.tsa.stattools as sts

In [47]:
for col in cols:
    test_stat, p_value, lags, obs, crit_vals, ic = sts.adfuller(df_comb[col].dropna())
    print(str(col))
    print('Test statistic:\t\t' + str(test_stat))
    print('Critical values:\t' + str(crit_vals))
    print('P-value:\t\t' + str(p_value))
    print('\n\n')

7d_roll_avg_norm_death
Test statistic:		-3.011001369305491
Critical values:	{'1%': -3.49181775886872, '5%': -2.8884437992971588, '10%': -2.5811201893779985}
P-value:		0.03387970538879231



7d_roll_avg_norm_positive
Test statistic:		-1.8079723198152358
Critical values:	{'1%': -3.4865346059036564, '5%': -2.8861509858476264, '10%': -2.579896092790057}
P-value:		0.3765109032255625



7d_roll_avg_norm_BA_ret
Test statistic:		-3.198247732167936
Critical values:	{'1%': -3.49181775886872, '5%': -2.8884437992971588, '10%': -2.5811201893779985}
P-value:		0.020086331671170687



7d_roll_avg_norm_SPY_ret
Test statistic:		-3.305162758716354
Critical values:	{'1%': -3.49181775886872, '5%': -2.8884437992971588, '10%': -2.5811201893779985}
P-value:		0.014649270391207737





With a 5% level of significance, we observe that three columns in the selected columns are from a stationary series of data.

- 7d_roll_avg_norm_death
- 7d_roll_avg_norm_BA_ret
- 7d_roll_avg_norm_SPY_ret

In [52]:
df_sel = df_comb.copy()[[
    '7d_roll_avg_norm_death',
    '7d_roll_avg_norm_BA_ret',
    '7d_roll_avg_norm_SPY_ret',
    'BA',
    'SPY'
]]

df_sel.dropna().head()

,7d_roll_avg_norm_death,7d_roll_avg_norm_BA_ret,7d_roll_avg_norm_SPY_ret,BA,SPY
Date,,,,,
2020-04-09,162.540625,-2.091723,-25.184701,151.839996,273.809021
2020-04-13,191.959870,-12.883485,-36.342339,147.330002,271.309113
2020-04-14,196.368518,-13.735669,-39.359599,141.000000,279.310822
2020-04-15,203.914088,-16.625346,-36.416229,145.979996,273.375977
2020-04-16,206.132542,11.256583,-17.536959,134.240005,274.694824


In [53]:
df_sel.corr()

,7d_roll_avg_norm_death,7d_roll_avg_norm_BA_ret,7d_roll_avg_norm_SPY_ret,BA,SPY
7d_roll_avg_norm_death,1.000000,0.052743,-0.305913,-0.748761,-0.669721
7d_roll_avg_norm_BA_ret,0.052743,1.000000,0.592298,-0.348309,-0.003910
7d_roll_avg_norm_SPY_ret,-0.305913,0.592298,1.000000,-0.039820,0.166964
BA,-0.748761,-0.348309,-0.039820,1.000000,0.623246
SPY,-0.669721,-0.003910,0.166964,0.623246,1.000000


In [5]:
ba_only = ba[['BA']]
ba_only

,BA
Date,
2020-03-31,149.139999
2020-04-01,130.699997
2020-04-02,123.269997
2020-04-03,124.519997
2020-04-06,148.770004
...,...
2020-09-24,146.050003
2020-09-25,156.029999
2020-09-28,166.080002


In [6]:
from pmdarima.arima import auto_arima

In [7]:
model_auto = auto_arima(ba_only.BA)

In [8]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  128
Model:               SARIMAX(0, 1, 0)   Log Likelihood                -448.054
Date:                Tue, 13 Apr 2021   AIC                            898.108
Time:                        11:26:13   BIC                            900.952
Sample:                             0   HQIC                           899.264
                                - 128                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        67.9039      5.449     12.461      0.000      57.224      78.584
===================================================================================
Ljung-Box (Q):                       48.89   Jarque-Bera (JB):                46.28
Prob(Q):                              0.16   Prob(JB):                         0.00
Heteroskedasticity (H):               0.37   Skew:                             0.36
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.87
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [9]:
size = int(len(ba_only)*0.8)
df_train = ba_only.iloc[:size]
df_test = ba_only.iloc[size:]

In [ ]:
model_auto = auto_arima(df.BA)

In [37]:
df_comb = ba.join(covid)
df_comb

,BA,SPY,deathIncrease
Date,,,
2020-03-31,149.139999,253.681808,890
2020-04-01,130.699997,242.264893,1011
2020-04-02,123.269997,247.855255,1172
2020-04-03,124.519997,244.272705,1286
2020-04-06,148.770004,260.679565,1318
...,...,...,...
2020-09-24,146.050003,321.075287,921
2020-09-25,156.029999,326.266083,844
2020-09-28,166.080002,331.685181,257


In [38]:
df_comb['rolling_7dd'] = df_comb.deathIncrease.rolling(window=7).mean()
df_comb

,BA,SPY,deathIncrease,rolling_7dd
Date,,,,
2020-03-31,149.139999,253.681808,890,NaN
2020-04-01,130.699997,242.264893,1011,NaN
2020-04-02,123.269997,247.855255,1172,NaN
2020-04-03,124.519997,244.272705,1286,NaN
2020-04-06,148.770004,260.679565,1318,NaN
...,...,...,...,...
2020-09-24,146.050003,321.075287,921,883.571429
2020-09-25,156.029999,326.266083,844,832.714286
2020-09-28,166.080002,331.685181,257,746.142857


In [32]:
df_comb = df_comb.pct_change(1)
df_comb

,BA,SPY,deathIncrease
Date,,,
2020-03-31,NaN,NaN,NaN
2020-04-01,-0.123642,-0.045005,0.135955
2020-04-02,-0.056848,0.023075,0.159248
2020-04-03,0.010140,-0.014454,0.097270
2020-04-06,0.194748,0.067166,0.024883
...,...,...,...
2020-09-24,-0.033933,0.002665,-0.205349
2020-09-25,0.068333,0.016167,-0.083605
2020-09-28,0.064411,0.016609,-0.695498


In [46]:
size = int(len(df_comb)*0.8)
df_train = df_comb.iloc[:size]
df_test = df_comb.iloc[size:]
df_train

,BA,SPY,deathIncrease,rolling_7dd
Date,,,,
2020-03-31,149.139999,253.681808,890,NaN
2020-04-01,130.699997,242.264893,1011,NaN
2020-04-02,123.269997,247.855255,1172,NaN
2020-04-03,124.519997,244.272705,1286,NaN
2020-04-06,148.770004,260.679565,1318,NaN
...,...,...,...,...
2020-08-18,170.229996,334.761780,1196,1036.142857
2020-08-19,169.270004,333.367920,1420,1178.142857
2020-08-20,169.580002,334.405914,1134,1150.714286


In [40]:
model_auto = auto_arima(df_train.BA.dropna())
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  102
Model:               SARIMAX(0, 1, 0)   Log Likelihood                -364.346
Date:                Tue, 13 Apr 2021   AIC                            730.692
Time:                        11:48:09   BIC                            733.307
Sample:                             0   HQIC                           731.751
                                - 102                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        79.5909      7.608     10.461      0.000      64.679      94.503
===================================================================================
Ljung-Box (Q):                       46.50   Jarque-Bera (JB):                23.77
Prob(Q):                              0.22   Prob(JB):                         0.00
Heteroskedasticity (H):               0.31   Skew:                             0.30
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [49]:
model_auto_ex = auto_arima(
    df_train.dropna().BA.pct_change(1)[1:],
    exogenous= df_train.dropna()[['SPY', 'rolling_7dd']][1:]
)

In [50]:
model_auto_ex.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   95
Model:                        SARIMAX   Log Likelihood                 148.303
Date:                Tue, 13 Apr 2021   AIC                           -288.606
Time:                        11:53:27   BIC                           -278.390
Sample:                             0   HQIC                          -284.478
                                 - 95                                         
Covariance Type:                  opg                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
intercept      -0.0001      0.159     -0.001      0.999      -0.312       0.312
SPY          3.088e-05      0.000      0.063      0.950      -0.001       0.001
rolling_7dd -4.869e-06   1.41e-05     -0.346      0.729   -3.24e-05    2.27e-05
sigma2          0.0026      0.000      8.552      0.000       0.002       0.003
===================================================================================
Ljung-Box (Q):                       64.37   Jarque-Bera (JB):                16.31
Prob(Q):                              0.01   Prob(JB):                         0.00
Heteroskedasticity (H):               0.29   Skew:                             0.60
Prob(H) (two-sided):                  0.00   Kurtosis:                         4.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
sns.set()

In [52]:
# Download data
data = yf.download("BA SPY", start="2020-03-31", end="2020-10-01")
stocks = data['Adj Close'][:]
stocks

[*********************100%***********************]  2 of 2 completed


,BA,SPY
Date,,
2020-03-31,149.139999,253.681808
2020-04-01,130.699997,242.264893
2020-04-02,123.269997,247.855255
2020-04-03,124.519997,244.272705
2020-04-06,148.770004,260.679565
...,...,...
2020-09-24,146.050003,321.075287
2020-09-25,156.029999,326.266083
2020-09-28,166.080002,331.685181


In [53]:
# Load COVID Data
covid_data = pd.read_csv('../ols-stocks-covid/all-states-history.csv', parse_dates=True, index_col=0)
covid_data

,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
date,,,,,,,,,,,,,,,,,,,,,
2020-10-02,WY,B,53.0,NaN,0,NaN,280.0,280.0,32.0,6,...,103278.0,1983,NaN,NaN,NaN,NaN,102353.0,2095,165242.0,1454
2020-10-02,NE,A,493.0,NaN,15,NaN,2356.0,2356.0,227.0,7,...,464264.0,4722,NaN,NaN,NaN,NaN,464576.0,4731,634687.0,9733
2020-10-02,ND,B,191.0,188.0,0,3.0,902.0,902.0,111.0,18,...,626450.0,7119,9731.0,NaN,NaN,NaN,244312.0,1412,649885.0,7432
2020-10-02,NC,A+,3608.0,3579.0,29,29.0,NaN,NaN,921.0,0,...,3099704.0,36043,NaN,3063.0,NaN,NaN,NaN,0,3094417.0,35876
2020-10-02,MT,C,186.0,NaN,5,NaN,729.0,729.0,177.0,2,...,353362.0,4653,NaN,NaN,NaN,NaN,NaN,0,353362.0,4653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-24,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
2020-01-23,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,0.0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
2020-01-23,MA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,2.0,1,NaN,NaN,NaN,NaN,NaN,0,2.0,1


In [54]:
covid = covid_data.groupby('date').sum()[['deathIncrease']]

start_date = '2020-03-31'
end_date = '2020-10-01'

covid = covid.loc[start_date:end_date]
covid

,deathIncrease
date,
2020-03-31,890
2020-04-01,1011
2020-04-02,1172
2020-04-03,1286
2020-04-04,1482
...,...
2020-09-27,307
2020-09-28,257
2020-09-29,739


In [55]:
df_comb = stocks.join(covid)
df_comb

,BA,SPY,deathIncrease
Date,,,
2020-03-31,149.139999,253.681808,890
2020-04-01,130.699997,242.264893,1011
2020-04-02,123.269997,247.855255,1172
2020-04-03,124.519997,244.272705,1286
2020-04-06,148.770004,260.679565,1318
...,...,...,...
2020-09-24,146.050003,321.075287,921
2020-09-25,156.029999,326.266083,844
2020-09-28,166.080002,331.685181,257


In [61]:
df_comb

,BA,SPY,deathIncrease
Date,,,
2020-03-31,149.139999,253.681808,890
2020-04-01,130.699997,242.264893,1011
2020-04-02,123.269997,247.855255,1172
2020-04-03,124.519997,244.272705,1286
2020-04-06,148.770004,260.679565,1318
...,...,...,...
2020-09-24,146.050003,321.075287,921
2020-09-25,156.029999,326.266083,844
2020-09-28,166.080002,331.685181,257


In [62]:
df_comb.to_csv('./clean_stocks_and_covid.csv')